In [14]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import faiss

In [15]:
# Étape 1 : Définir les paragraphes
# Nom du fichier texte à lire
size = "100k"
file_path = f"paragraphs_{size}.txt"

# Initialisation de la liste pour stocker les paragraphes
paragraphs = []

# Ouvrir le fichier en mode lecture
with open(file_path, "r", encoding="utf-8") as f:
    # Lire chaque ligne du fichier
    for ligne in f:
        # Ajouter la ligne à la liste des paragraphes
        paragraphs.append(ligne.strip())  # strip() pour enlever les espaces superflus et les sauts de ligne

In [16]:
# Test
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(paragraphs, show_progress_bar=True, normalize_embeddings=True).astype('float32')
np.save(f'embeddings_{size}.npy', embeddings)

Batches:   0%|          | 0/3328 [00:00<?, ?it/s]

In [17]:
embeddings = np.load(f'embeddings_{size}.npy')

In [18]:
# k-means params
k = int(embeddings.shape[0]/39)
d = embeddings.shape[1]
res = faiss.StandardGpuResources()  # Créer les ressources GPU

In [19]:
# Convertir les données en float32 si ce n'est pas déjà le cas
embeddings = embeddings.astype(np.float32)

# Configurer l'index GPU
flat_config = faiss.GpuIndexFlatConfig()
flat_config.device = 0  # GPU 0

index = faiss.GpuIndexFlatL2(res, d, flat_config)

# Clustering
clustering = faiss.Clustering(d, k)
clustering.train(embeddings, index)

# Récupérer les centroïdes
centroids = faiss.vector_to_array(clustering.centroids).reshape(k, d)
np.save(f'centroids_{size}.npy', centroids)

In [20]:
centroids = np.load(f'centroids_{size}.npy')

50k:  
8.2s

100k:  
11.5s

200k:  


In [21]:
# Paramètres
d = embeddings.shape[1]  # Dimension des embeddings
nlist = 100  # Nombre de cellules de partition pour IVF, à ajuster en fonction de la taille des données
M = 32  # Nombre de sous-vecteurs pour PQ, à ajuster en fonction de la dimension des embeddings
n_bits = 8  # Nombre de bits pour la quantification, généralement entre 8 et 16

# Créer un index de base (par exemple, IndexFlatL2 pour la distance euclidienne)
faiss_index_base = faiss.IndexFlatL2(d)

# Créer l'index IVF avec PQ
quantizer = faiss.IndexFlatL2(d)  # Quantiseur pour l'index IVF
index_ivf = faiss.IndexIVFPQ(quantizer, d, nlist, M, n_bits)

# Vérifier si l'index IVF est déjà entraîné
if not index_ivf.is_trained:
    # Entraîner l'index avec un sous-ensemble d'embeddings (par exemple, un échantillon aléatoire)
    index_ivf.train(embeddings)

In [ ]:
# Convertir l'index IVF+PQ en un index GPU, can't use, range_search not inplemented for GPU
#gpu_index = faiss.index_cpu_to_gpu(res, 0, index_ivf)  # 0 pour utiliser le GPU 0

In [ ]:
# Ajouter les embeddings à l'index
index_ivf.add(embeddings)  # Ajouter les embeddings à l'index

: 

50k:  


100k:  
43.6s

200k:  


In [24]:
def find_similar_paragraphs_gpu(index_ivf, embeddings, threshold=0.75, res=res):
    """
    Recherche tous les voisins proches dont la similarité est >= threshold
    (en utilisant la méthode range_search de FAISS), sur le GPU.
    """
    # Assurer que les embeddings sont en float32
    embeddings = embeddings.astype(np.float32)
    limits, distances, indices = index_ivf.range_search(embeddings, threshold)
    return limits, distances, indices

limits, distances, indices = find_similar_paragraphs_gpu(index_ivf, embeddings, threshold=0.75)

: 

: 

50k, IndexIVFPQ:  
1.8s

100k, IndexIVFPQ:  


200k, IndexIVFPQ:  


In [ ]:
#np.save(f'limits_{size}.npy', limits)
#np.save(f'distances_{size}.npy', distances)
#np.save(f'indices_{size}.npy', indices)

In [ ]:
for i in range(len(embeddings)-1, len(embeddings)):
    print(f"Vecteur {i}:")
    start = limits[i]  # Début des voisins du i-ème vecteur
    end = limits[i+1]  # Fin des voisins du i-ème vecteur
    for j in range(start, end):
        if indices[j] != i:  # Ne pas inclure le vecteur lui-même
            print(f"  Voisin {indices[j]} avec une distance de {distances[j]:.4f}")
    print()


Vecteur 53247:
  Voisin 12 avec une distance de 0.0000
  Voisin 25 avec une distance de 0.0000
  Voisin 38 avec une distance de 0.0000
  Voisin 51 avec une distance de 0.0000
  Voisin 64 avec une distance de 0.0000
  Voisin 77 avec une distance de 0.0000
  Voisin 90 avec une distance de 0.0000
  Voisin 103 avec une distance de 0.0000
  Voisin 116 avec une distance de 0.0000
  Voisin 129 avec une distance de 0.0000
  Voisin 142 avec une distance de 0.0000
  Voisin 155 avec une distance de 0.0000
  Voisin 168 avec une distance de 0.0000
  Voisin 181 avec une distance de 0.0000
  Voisin 194 avec une distance de 0.0000
  Voisin 207 avec une distance de 0.0000
  Voisin 220 avec une distance de 0.0000
  Voisin 233 avec une distance de 0.0000
  Voisin 246 avec une distance de 0.0000
  Voisin 259 avec une distance de 0.0000
  Voisin 272 avec une distance de 0.0000
  Voisin 285 avec une distance de 0.0000
  Voisin 298 avec une distance de 0.0000
  Voisin 311 avec une distance de 0.0000
  Voisin